In [1]:
# Import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# !pip install geocoder
# print('DONE')

In [3]:
# %reload_ext autoreload
# %autoreload 2 # autoreload reloads modules automatically before entering the execution of code typed.
%matplotlib inline

In [4]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code # A status_code of 200 means that the page downloaded successfully. 

200

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
table  = soup.find('table', { "class" : "wikitable sortable"})

# Transform the data into a pandas dataframe
1. **We will work on the columns first**

In [7]:
header_rows  = table.find_all('th')
header = [column_name.get_text().rstrip() for column_name in header_rows] # use rstrip() to remove all the trailing newline
header

['Postcode', 'Borough', 'Neighborhood']

2. **We proceed then on our data**
    1. Ignore cells with a borough that is Not assigned.
    2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
tr = table.findAll(['tr'])
data = []
for cell in tr:
    row    = cell.find_all('td')
    if len(row) == 0: # skip emtpy row
        continue
    else:
        if row[1].text == 'Not assigned': # Ignore cells with a borough that is "Not assigned". 
            continue
        else:
            if row[2].text.rstrip()  == 'Not assigned': # If a cell has a borough but that cell has a "Not assigned" neighborhood
                row[2] = row[1]                                   # then the neighborhood will be the same as the borough.
    df_row = [x.get_text().rstrip() for x in row]
    data.append(df_row)    

In [9]:
df =  pd.DataFrame(data=data,columns=header)
df.columns = ['PostalCode', 'Borough', 'Neighborhood'] # The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


3. **Combined same postal code area into one row**

In [11]:
df_grouped = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df_grouped.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


4. **Use the .shape method to print the number of rows of your dataframe.**

In [54]:
df_grouped.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood anbd merge

In [34]:
# Since the geocoder API didnt work,  we will use the CSV
path = 'http://cocl.us/Geospatial_data'
df_lon_lat = pd.read_csv(path)

In [37]:
df_lon_lat.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
df_lon_lat.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [52]:
df_2 =pd.merge(df, df_lon_lat, on='PostalCode')
df_2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
